This notebook can be used to reproduce any of the experiments that were conducted to obtain the empirical scaling law for image denoising in _Section 3: Empirical scaling laws for denoising_ from the paper **Scaling Laws For Deep Learning Based Image Reconstruction**

In [ ]:
import os
import numpy as np
import json

from utils.main_function_helpers import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Specify which experiments to run by indicating training set size and network size. Corresponding hyperparameters are loaded automatically.

For all available combinations of training set and network size see the config files in options/

In [ ]:
#######################################################
# Adjust the following parameters
#######################################################
# Start or continue training
training = True
# Evaluate last and best checkpoint on validation set
val_testing = True
# Evaluate last and best checkpoint on test set
test_testing = True


# Assign an ID to the experiment
exp_nums = ['001','002'] 
# Path to ImageNet train directory
path_to_ImageNet_train = '../../../media/hdd1/ImageNet/ILSVRC/Data/CLS-LOC/'
# training set size
train_sizes = [300,3000]
# network size defined by the number of channels in the first layer
channels = [64,128]

########################################################

# Sanity checks
if len(train_sizes) != len(exp_nums) or len(channels) != len(exp_nums):
    raise ValueError("Specify experiment ID for each experiment") 

Load hyperparameter configurations for each experiment from options/

In [ ]:
hps = []
for train_size,channel in zip(train_sizes, channels):
    options_name = "options/trainsize{}_channels{}.txt".format(train_size,channel)

    # Load hyperparameter options
    with open(options_name) as handle:
        hp = json.load(handle)
    hp['path_to_ImageNet_train'] = [path_to_ImageNet_train]
    hps.append(hp)

Run training/testing

In [ ]:
for ee in range(len(exp_nums)):
    
    hp = hps[ee]
    num_runs = list(np.arange(hp['num_runs'][0]))
    
    for rr in num_runs:
        exp_name =  'E' + exp_nums[ee] + \
                    '_t' + str(hp['train_size'][0]) + \
                    '_l' + str(hp['num_pool_layers'][0]) + \
                    'c' + str(hp['chans'][0]) + \
                    '_bs' + str(hp['batch_size'][0]) +\
                    '_lr' + str(hp['lr'][0])[2:]
        if rr>0:
            exp_name = exp_name + '_run{}'.format(rr+1)
        if not os.path.isdir('./'+exp_name):
            os.mkdir('./'+exp_name)
        
        ########
        # Training
        ########
        if training:  
            print('\n{} - Training\n'.format(exp_name))
            args = get_args(hp,rr)
            args.output_dir = './'+exp_name
            cli_main(args)
            print('\n{} - Training finished\n'.format(exp_name))
            
        ########
        # Testing
        ########
        if val_testing or test_testing:
            print('\n{} - Testing\n'.format(exp_name))

            test_modes = []
            if val_testing:
                test_modes.append("val")
            if test_testing:
                test_modes.append("test")

            for test_mode in test_modes:
                for restore_mode in ["last","best"]:
                    args = get_args(hp,rr)
                    args.output_dir = './'+exp_name
                    args.restore_mode = restore_mode
                    args.test_mode = test_mode
                    args.test_noise_std_min = args.noise_std
                    args.test_noise_std_max = args.noise_std
                    cli_main_test(args)

            print('\n{} - Testing finished\n'.format(exp_name))           